In [1]:
from MLRF.datatools import h5kit, psdkit
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.signal import find_peaks
from random import shuffle

In [2]:
wifi_iq = h5kit('data/2_4ghz_indoor.h5')
bt_iq = h5kit('data/2_4ghz_bluetooth.h5')
psd_test = h5kit('psd_test.h5')
psd_train = h5kit('psd_train.h5')

In [3]:
# Process WiFi data
keys = wifi_iq.keys()[0]
data = wifi_iq.read(keys).ravel()
data = psdkit.normalize(data)
data = psdkit.deinterlace_iq(data)
split_index = int(0.7 * len(data) // 2048)

# for i, idx in enumerate(tqdm(range(split_index), desc="wifi iq -> psd (Training)")):
#     segment = data[idx * 2048 : (idx + 1) * 2048]
#     psd = psdkit.iq_to_psd(segment, 30000000, 2437000000).astype(np.float32)
#     psd_train.write(f"wifi_{i}", psd)

# for i, idx in enumerate(
#     tqdm(range(split_index, len(data) // 2048), desc="wifi iq -> psd (Testing)")
# ):
#     segment = data[idx * 2048 : (idx + 1) * 2048]
#     psd = psdkit.iq_to_psd(segment, 30000000, 2437000000).astype(np.float32)
#     psd_test.write(f"wifi_{i}", psd)

NOISE_THRESHOLD = -85.0
PEAK_PROMINENCE = 2
# Process Bluetooth data
keys = bt_iq.keys()
shuffle(keys)
keys = keys[:28]
data = bt_iq.read(keys).ravel()
data = psdkit.normalize(data)
data = psdkit.deinterlace_iq(data)
split_index = int(0.7 * len(data) // 2048)


for i, idx in enumerate(
    tqdm(range(split_index), desc="bluetooth iq -> psd (Training)")
):
    segment = data[idx * 2048 : (idx + 1) * 2048]
    psd = psdkit.iq_to_psd(segment, 30000000, 2437000000).astype(np.float32)
    peaks, properties = find_peaks(
        psd, height=NOISE_THRESHOLD, prominence=PEAK_PROMINENCE
    )
    if len(peaks) > 0:
        psd_train.write(f"bluetooth_{i}", psd)

for i, idx in enumerate(
    tqdm(range(split_index, len(data) // 2048), desc="bluetooth iq -> psd (Testing)")
):
    segment = data[idx * 2048 : (idx + 1) * 2048]
    psd = psdkit.iq_to_psd(segment, 30000000, 2437000000).astype(np.float32)
    peaks, properties = find_peaks(
        psd, height=NOISE_THRESHOLD, prominence=PEAK_PROMINENCE
    )
    if len(peaks) > 0:
        psd_test.write(f"bluetooth_{i}", psd)

bluetooth iq -> psd (Training):   0%|          | 0/287109 [00:00<?, ?it/s]

bluetooth iq -> psd (Testing):   0%|          | 0/123047 [00:00<?, ?it/s]

In [4]:
keys = psd_train.keys()
print(len([key for key in keys if 'bluetooth' in key]))
print(len([key for key in keys if 'wifi' in key]))

9447
10253
